# Unit Test Generator

This notebook generates unit tests for code.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
CODEQWEN_MODEL = "codeqwen"
DEEPSEEK_MODEL = "deepseek-r1"
LLAMA_MODEL = "llama3.2"
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='dummy')

In [ ]:
system_prompt = "You are a coding assistant who generates unit tests for given code. "
system_prompt += "Your response should include only the generated unit tests. Do not include summary text or explanations in your response."

In [ ]:
def get_user_prompt(code):
    user_prompt = f"Generate unit tests for the following code.\n\n{code}"
    return user_prompt

In [ ]:
def get_messages(code):
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': get_user_prompt(code)}
    ]
    return messages

In [ ]:
def generate_tests(code, model):
    model_name = ""
    
    match model:
        case 'CodeQwen':
            model_name = CODEQWEN_MODEL
        case 'DeepSeek':
            model_name = DEEPSEEK_MODEL
        case 'Llama':
            model_name = LLAMA_MODEL
            
    stream = ollama.chat.completions.create(model=model_name, messages=get_messages(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## Unit Test Generator")
    with gr.Row():
        code = gr.Textbox(label="Code:", value=pi, lines=10)
        tests = gr.Textbox(label="Unit tests:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["CodeQwen", "DeepSeek", "Llama"], label="Select model", value="CodeQwen")
        comment = gr.Button("Generate unit tests")

    comment.click(generate_tests, inputs=[code, model], outputs=[tests])

ui.launch(inbrowser=True)